<a href="https://colab.research.google.com/github/UCREL/Session1_Visualisation_and_Summarisation/blob/main/NoteBooks/3_Text_Summariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tutorial: Building a Text Summarizer
![Comparison Diagram](https://raw.githubusercontent.com/Nouran-Khallaf/NLP_Visualisation/main/static/images/Extractive-vs-Abstractive-Summarization.png)
*Diagram 1: Comparative Analysis of summarisation Techniques*


### Introduction
This notebook demonstrates how to build a text summariser using  Multi-document Summaries Corpora (2011 and 2013) dataset.
The dataset is derived from publicly available WikiNews (http://www.wikinews.org/) English texts. The source texts were under CC Attribution Licence V2.5 (http://creativecommons.org/licenses/by/2.5/). Texts in other languages have been translated by native speakers of each language.
Languages included:
- Arabic
- Czech
- English
- French
- Greek
- Hebrew
- Hindi
- Chinese (2013)
- Romanian (2013)
- Spanish (2013)

For more information, please visit [Multi-document Summaries Corpora](https://www.lancaster.ac.uk/staff/elhaj/corpora.html).



**Extractive summarisation**:
Extractive summarisation involves selecting key sentences or phrases from the original text and concatenating them to create a summary. The goal is to extract the most important information while maintaining the original context. This method does not generate new sentences but rather identifies and extracts the most relevant parts of the text.




**Abstractive summarisation**:
Abstractive summarisation, on the other hand, involves generating new sentences that convey the most important information from the original text. This method goes beyond simply extracting key phrases and instead uses techniques like natural language generation to create a coherent and concise summary that may include paraphrasing and restructuring of information.

![Comparison Diagram](https://raw.githubusercontent.com/Nouran-Khallaf/NLP_Visualisation/main/static/images/extractive.webp)![Comparison Diagram](https://raw.githubusercontent.com/Nouran-Khallaf/NLP_Visualisation/main/static/images/abstractive.png)
*Diagram 2: Analysis of Extractive summarisation *
*Diagram 3: Analysis of Abstractive summarisation *

---


In [ ]:
!pip install gdown
!pip install plotly
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install tabulate
!pip install summa

## Step 1: Load and Prepare Data
We will load the dataset and clean it. For demonstration purposes.

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=1GSRJgJaRzb1OauKtHdwNoF4UnkKAt9cK'
output = 'Multilingual-Dataset.zip'
gdown.download(url, output, quiet=False)

In [ ]:
import zipfile
import os

if zipfile.is_zipfile(output):
    with zipfile.ZipFile(output, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    print("Files extracted:")
    print(os.listdir('/content/'))
else:
    print("Downloaded file is not a zip file.")


## **Data** **Prepration**
we need to organize the text files in the `SourceTexts` directory into subdirectories based on language identifiers extracted from the filenames. It dynamically identifies unique language codes, creates corresponding language-specific folders, and moves each file to its appropriate folder, thereby streamlining file management and access based on language.

- Arabic
- Czech
- English
- French
- Greek
- Hebrew
- Hindi

In [ ]:
import os
import shutil

# Define the source directory
source_dir = 'SourceTexts'

# List all files in the source directory
files = os.listdir(source_dir)

# Create a set to store unique language identifiers
languages = set()

# Identify unique languages from filenames
for file in files:
    language = file.split('.')[1].lower()  # Normalize to lowercase
    languages.add(language)

# Create folders for each language and move corresponding files
for language in languages:
    # Create the language folder if it doesn't exist
    language_dir = os.path.join(source_dir, language)
    os.makedirs(language_dir, exist_ok=True)

    # Move files into the language folder
    for file in files:
        if file.split('.')[1].lower() == language:  # Normalize to lowercase
            shutil.move(os.path.join(source_dir, file), os.path.join(language_dir, file))

print("Files have been organized into folders.")


Here we combine the content of text files from the `SourceTexts` directory into a single Pandas DataFrame, with each file's content stored as a row in the DataFrame. It then saves the DataFrame to a CSV file.

1. **Import Required Modules**:
   - The script imports the `os` module for directory and file operations and the `pandas` module for creating and manipulating the DataFrame.

In [ ]:
import os
import pandas as pd


2. **Define the Function to Combine Files**:
   - A function named `combine_files_to_dataframe` is defined. This function takes a directory path as input and returns a DataFrame containing the file names and their contents.
- **Initialize an Empty List**: An empty list named `data` is created to store the file names and their contents.
- **Loop Through Files**: The function loops through all files in the specified directory using `os.listdir()`.
- **Construct Full File Path**: For each file, the full path is constructed using `os.path.join()`.
- **Check if it’s a File**: The script ensures the path is a file using `os.path.isfile()`.
- **Read File Content**: If it’s a file, the script reads its content and appends a dictionary containing the file name (`Title`) and its content (`Content`) to the `data` list.
- **Create a DataFrame**: Finally, a Pandas DataFrame is created from the `data` list and returned.

In [ ]:
def combine_files_to_dataframe(directory_path):
       # Initialize an empty list to store the data
       data = []

       # Loop through all files in the directory
       for file_name in os.listdir(directory_path):
           # Construct full file path
           file_path = os.path.join(directory_path, file_name)

           # Ensure it's a file
           if os.path.isfile(file_path):
               # Read the content of each file
               with open(file_path, 'r') as file:
                   content = file.read()
                   # Append the file name and content to the data list
                   data.append({'Title': file_name, 'Content': content.strip()})

       # Create a DataFrame from the data list
       df = pd.DataFrame(data)
       return df


3. **Specify the Directory Path**:
   - The directory containing the files is specified. In this example, the directory path is `/content/SourceTexts/english`.

In [ ]:
directory_path = '/content/SourceTexts/english' ## here you can change the language

4. **Combine Files into a DataFrame**:
   - The `combine_files_to_dataframe` function is called with the specified directory path, and the resulting DataFrame is stored in the variable `df`.

In [ ]:
df = combine_files_to_dataframe(directory_path)

5. **Display the DataFrame**:
   - The DataFrame is printed to the console to display the combined content of the files.

In [ ]:
print(df)



6. **Save the DataFrame to a CSV File**:
   - The DataFrame is saved to a CSV file named `combined_files.csv` without the index using the `to_csv` method.

In [ ]:
df.to_csv('combined_files.csv', index=False)


---

> The following code cell is designed to perform the combined operations described above. Let's execute it to see the results.
---

In [ ]:
import os
import pandas as pd

def combine_files_to_dataframe(directory_path):
    # Initialize an empty list to store the data
    data = []

    # Loop through all files in the directory
    for file_name in os.listdir(directory_path):
        # Construct full file path
        file_path = os.path.join(directory_path, file_name)

        # Ensure it's a file
        if os.path.isfile(file_path):
            # Read the content of each file
            with open(file_path, 'r') as file:
                content = file.read()
                # Append the file name and content to the data list
                data.append({'Title': file_name, 'Content': content.strip()})

    # Create a DataFrame from the data list
    df = pd.DataFrame(data)
    return df

# Specify the directory containing the files
directory_path = '/content/SourceTexts/english'

# Combine the files into a DataFrame
df = combine_files_to_dataframe(directory_path)

# Display the DataFrame
print(df)

# save the DataFrame to a CSV file
df.to_csv('combined_files.csv', index=False)

## Step 2: Data Cleaning

Enhances the readability and usability of text data by:
1. Removing HTML tags using the `BeautifulSoup` library.
2. Cleaning the text with regular expressions to replace multiple spaces, remove standalone numbers, and trim whitespace.
3. Loading the original text data from a CSV file into a DataFrame.
4. Applying the text cleaning process to the 'Content' column and storing the cleaned text in a new column named 'clean_content'.

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup

2. **Define a Function to Clean HTML**:
   - A function named `clean_html` is defined to remove HTML tags from a string. It uses `BeautifulSoup` to parse the HTML and extract the text content.
- **Parameter**: `raw_html` - The raw HTML string to be cleaned.
- **Processing**: `BeautifulSoup` parses the HTML and the `get_text()` method extracts the text content.
- **Return**: The cleaned text without HTML tags.

In [ ]:
def clean_html(raw_html):
       """Remove HTML tags from a string"""
       soup = BeautifulSoup(raw_html, "html.parser")
       return soup.get_text()

3. **Define a Function to Clean Text**:
  - A function named `clean_text` is defined to clean the text by removing unwanted patterns and extra spaces. It first removes HTML tags using the `clean_html` function and then applies various regular expressions to clean the text further.
- **Parameter**: `text` - The text string to be cleaned.
- **Processing**:
     - `clean_html(text)`: Removes HTML tags.
     - `re.sub(r'\s+', ' ', text)`: Replaces multiple spaces and newlines with a single space.
     - `re.sub(r'\b\d+\b', '', text)`: Removes standalone numbers (optional, based on specific needs).
     - `text.strip()`: Removes leading and trailing spaces.
- **Return**: The cleaned text.

In [ ]:
def clean_text(text):
       """Remove unwanted text patterns and clean the text"""
       text = clean_html(text)
       text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
       text = re.sub(r'\b\d+\b', '', text)  # Remove standalone numbers if needed
       text = text.strip()  # Remove leading/trailing spaces
       return text

4. **Load the DataFrame**:
   - The script loads a DataFrame from a CSV file named `combined_files.csv` using the `pandas` library.


In [ ]:
df = pd.read_csv('combined_files.csv')

5. **Apply Preprocessing to the 'Content' Column**:
   - The script applies the `clean_text` function to each entry in the 'Content' column of the DataFrame, creating a new column 'clean_content' with the cleaned text.
- **Processing**: The `apply` method is used to apply the `clean_text` function to each row in the 'Content' column.
- **Result**: A new column 'clean_content' is added to the DataFrame containing the cleaned text.

In [ ]:
df['clean_content'] = df['Content'].apply(clean_text)


---

> The following code cell is designed to perform the combined operations described above. Let's execute it to see the results.
---

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup

def clean_html(raw_html):
    """Remove HTML tags from a string"""
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text()

def clean_text(text):
    """Remove unwanted text patterns and clean the text"""
    text = clean_html(text)
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
    text = re.sub(r'\b\d+\b', '', text)  # Remove standalone numbers if needed
    text = text.strip()  # Remove leading/trailing spaces
    return text


# Load your DataFrame
df = pd.read_csv('combined_files.csv')

# Apply preprocessing to the 'Content' column
df['clean_content'] = df['Content'].apply(clean_text)

## Step 9: Extractive Summary

## 1- Extractive Summarisation using TF-IDF
extractive summarisation using the TF-IDF (Term Frequency-Inverse Document Frequency) method:

1. **Calculate TF-IDF scores for each term in the document**.
2. **Rank sentences based on the sum of TF-IDF scores of their terms**.
3. **Select the top N sentences with the highest scores as the summary**.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# Function to generate extractive summary using TF-IDF
def extractive_summary(text, top_n=3):
    sentences = text.split('.')
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    sentence_scores = np.sum(X.toarray(), axis=1)
    top_sentence_indices = sentence_scores.argsort()[-top_n:][::-1]
    summary = '. '.join([sentences[i].strip() for i in top_sentence_indices if sentences[i].strip()]) + '.'
    return summary

# Apply the summarisation function to each content
df['summary_tfidf'] = df['clean_content'].apply(extractive_summary)

# Display the DataFrame with summaries
print(df[['Title', 'clean_content','summary_tfidf']])

## Extractive Summarisation using Sumy and Summa
Implement a simple text summariser based on sentence extraction.

In [ ]:
!pip install sumy
!pip install nltk
!pip install summa
!pip install seaborn mglearn
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from summa import summarizer
nltk.download('punkt')

### summarising Text Using LSA summariser

utilize the Latent Semantic Analysis (LSA) summariser from the Sumy package to summarise text. The LSA summariser works by reducing the dimensionality of the text representation and selecting the most significant sentences to form a coherent summary.

1. **Define the summarisation Function**:
   - We create a function `summarise_text` that accepts the text to be summarised and the number of sentences desired in the summary.
   - This function utilizes the LSA summariser to generate the summary.

2. **Apply the summarisation Function**:
   - We apply the `summarise_text` function to the 'clean_content' column of our DataFrame.
   - The resulting summaries are stored in a new column called 'sumy_summary'.

3. **Display and Save the summarised Data**:
   - We display the DataFrame containing the new summaries.
   - Finally, we save the DataFrame with the summaries to a CSV file for further use or analysis.



In [ ]:
def summarise_text(text, sentences_count=2):
    """summarise the text using LSA summariser"""
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summariser = LsaSummarizer()
    summary = summariser(parser.document, sentences_count)
    return ' '.join([str(sentence) for sentence in summary])

# Apply summarisation to the 'clean_content' column
df['summary_sumy'] = df['clean_content'].apply(lambda x: summarise_text(x, sentences_count=2))

# Display the DataFrame with summaries
print(df[['Title', 'summary_sumy']])

# Save the DataFrame with summaries to a CSV file
df.to_csv('sumy_summary.csv', index=False)

### summarising Text Using the Summa Library

utilize the `summa` library to summarise text. The `summa` library leverages the TextRank algorithm to generate concise and effective summaries.

1. **Define the summarisation Function**:
   - We create a function `summa_summarise` that accepts the text to be summarised and a `ratio` parameter. The `ratio` determines the fraction of the original text to be retained in the summary.
   - This function uses the `summarizer.summarize` method from the `summa` library to perform the summarization.

2. **Apply the summarisation Function**:
   - We apply the `summa_summarize` function to the 'clean_content' column of our DataFrame.
   - The resulting summaries are stored in a new column called 'summa_summary'.

3. **Save the summarised Data**:
   - Finally, we save the DataFrame with the new summaries to a CSV file for further use or analysis.



In [ ]:
def summa_summarise(text, ratio=0.2):
    """summarise text using the summa library"""
    summary = summarizer.summarize(text, ratio=ratio, language='english')
    return summary
# Apply summarisation to the 'clean_content' column using summa
df['summary_summa'] = df['clean_content'].apply(lambda x: summa_summarise(x, ratio=0.2))
# Save the cleaned and summarised data to a CSV file if needed
df.to_csv('summa_summary.csv', index=False)

### Save summarised Data
Save the summarised data to a new CSV file.

In [ ]:
output_file = 'summarized_data.csv'
df.to_csv(output_file, index=False)
print(f"Summarized data saved to {output_file}")

##  Visualise Results


---

>To visualise the most frequent words in the summaries from each method using **word cloud**:
---

In [ ]:
df

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
summarisation_methods = ['summary_sumy', 'summary_summa', 'summary_tfidf']
# Generate word clouds for each summarisation method
for method in summarisation_methods:
    text = ' '.join(df[method])
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud for {method}')
    plt.axis('off')
    plt.show()


---

>Visualise the length of original texts vs. summaries.
---

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the data
df = pd.read_csv('summarized_data.csv')

# Calculating the lengths of the original and summarized texts
df['original_length'] = df['clean_content'].apply(lambda x: len(x.split()))
df['summary_sumy_length'] = df['summary_sumy'].apply(lambda x: len(x.split()))
df['summary_summa_length'] = df['summary_summa'].apply(lambda x: len(x.split()))
df['summary_tfidf_length'] = df['summary_tfidf'].apply(lambda x: len(x.split()))
#df['summary_transformer_length'] = df['summary_transformer'].apply(lambda x: len(x.split()))  # Assuming transformer summary column exists

# Plotting the distribution of text lengths
plt.figure(figsize=(14, 8))
sns.histplot(df['original_length'], kde=True, color='blue', label='Original Length', bins=30)
sns.histplot(df['summary_sumy_length'], kde=True, color='red', label='Sumy Summary Length', bins=30)
sns.histplot(df['summary_summa_length'], kde=True, color='green', label='Summa Summary Length', bins=30)
sns.histplot(df['summary_tfidf_length'], kde=True, color='purple', label='TFIDF Summary Length', bins=30)
#sns.histplot(df['summary_transformer_length'], kde=True, color='orange', label='Transformer Summary Length', bins=30)
plt.legend()
plt.title('Distribution of Text Lengths for Different summarisation Methods')
plt.xlabel('Length of Text')
plt.ylabel('Frequency')
plt.show()


### **Do you think of better visualisation?**









### 1. Box Plots
**Box Plots**:
   Box plots can show the distribution, median, and outliers for each summarisation method in a more compact way.

In [ ]:
plt.figure(figsize=(14, 8))
sns.boxplot(data=df[['original_length', 'summary_sumy_length', 'summary_summa_length', 'summary_tfidf_length']])
plt.title('Box Plot of Text Lengths for Different summarisation Methods')
plt.xlabel('Text Type')
plt.ylabel('Length of Text')
plt.xticks([0, 1, 2, 3, 4], ['Original', 'Sumy', 'Summa', 'TFIDF', 'Transformer'])
plt.show()


### 2. Bar Plots of Mean Lengths
**Bar Plots of Mean Lengths**:
   A bar plot can provide a quick comparison of the average lengths of the original texts and their summaries.


In [ ]:
mean_lengths = {
    'Original': df['original_length'].mean(),
    'Sumy': df['summary_sumy_length'].mean(),
    'Summa': df['summary_summa_length'].mean(),
    'TFIDF': df['summary_tfidf_length'].mean()
}

plt.figure(figsize=(14, 8))
plt.bar(mean_lengths.keys(), mean_lengths.values(), color=['blue', 'red', 'green', 'purple', 'orange'])
plt.title('Average Text Lengths for Different Summarization Methods')
plt.xlabel('Text Type')
plt.ylabel('Average Length of Text')
plt.show()



###3. Pairwise Comparison with Scatter Plots
**Pairwise Comparison with Scatter Plots**:
   Scatter plots can help visualize the relationship between the lengths of original texts and their summaries for each method.


In [ ]:
plt.figure(figsize=(14, 8))
plt.scatter(df['original_length'], df['summary_sumy_length'], color='red', label='Sumy', alpha=0.6)
plt.scatter(df['original_length'], df['summary_summa_length'], color='green', label='Summa', alpha=0.6)
plt.scatter(df['original_length'], df['summary_tfidf_length'], color='purple', label='TFIDF', alpha=0.6)
plt.title('Pairwise Comparison of Text Lengths')
plt.xlabel('Original Length of Text')
plt.ylabel('Summary Length of Text')
plt.legend()
plt.show()


### 4. Distribution Plot with KDE
**Distribution Plot with KDE**:
   A more detailed distribution plot using Kernel Density Estimation (KDE) to show the continuous distribution of text lengths.

In [ ]:
plt.figure(figsize=(14, 8))
sns.kdeplot(df['original_length'], color='blue', label='Original Length', fill=True)
sns.kdeplot(df['summary_sumy_length'], color='red', label='Sumy Summary Length', fill=True)
sns.kdeplot(df['summary_summa_length'], color='green', label='Summa Summary Length', fill=True)
sns.kdeplot(df['summary_tfidf_length'], color='purple', label='TFIDF Summary Length', fill=True)

plt.title('KDE Plot of Text Lengths for Different summarisation Methods')
plt.xlabel('Length of Text')
plt.ylabel('Density')
plt.legend()
plt.show()


### 5. Violin Plots
Violin plots combine aspects of box plots and KDE plots, providing a richer visualization of the data distribution.


In [ ]:

plt.figure(figsize=(14, 8))
sns.violinplot(data=df[['original_length', 'summary_sumy_length', 'summary_summa_length', 'summary_tfidf_length']])
plt.title('Violin Plot of Text Lengths for Different summarisation Methods')
plt.xlabel('Text Type')
plt.ylabel('Length of Text')
plt.xticks([0, 1, 2, 3, 4], ['Original', 'Sumy', 'Summa', 'TFIDF', 'Transformer'])
plt.show()


### 6. Heatmap of Correlations
A heatmap can show the correlation between the lengths of texts from different summarisation methods.


In [ ]:
plt.figure(figsize=(10, 8))
correlation_matrix = df[['original_length', 'summary_sumy_length', 'summary_summa_length', 'summary_tfidf_length']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', cbar=True)
plt.title('Correlation Heatmap of Text Lengths')
plt.show()

### 7. Paired Density and Scatter Plot (Pairplot)
Pairplots can show pairwise relationships and distributions of text lengths for different methods.



In [ ]:

sns.pairplot(df[['original_length', 'summary_sumy_length', 'summary_summa_length', 'summary_tfidf_length']], kind='scatter', diag_kind='kde')
plt.suptitle('Pairplot of Text Lengths for Different summarisation Methods', y=1.02)
plt.show()



### 8. Joint Plot
Joint plots provide a way to visually represent the relationship between two variables along with their distributions.


In [ ]:
plt.figure(figsize=(14, 8))
sns.jointplot(x='original_length', y='summary_sumy_length', data=df, kind='scatter', color='red', marginal_kws=dict(bins=30, fill=True))
plt.suptitle('Joint Plot of Original vs Sumy Summary Length', y=1.02)
plt.show()

### 9. 3D Scatter Plot
visualize three dimensions of the data (e.g., lengths of original, Sumy, and Summa summaries), a 3D scatter plot.


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['original_length'], df['summary_sumy_length'], df['summary_summa_length'], c='r', marker='o')
ax.set_xlabel('Original Length')
ax.set_ylabel('Sumy Summary Length')
ax.set_zlabel('Summa Summary Length')
plt.title('3D Scatter Plot of Text Lengths')
plt.show()

## Step 10: Abstarctive summary
Creating a summariser involves several steps, especially for abstractive summarisation:

1. **Text Preprocessing**: Clean and prepare the text by removing stop words, punctuation, and performing tokenization.
2. **Feature Extraction**: For extractive summarisation, extract features like sentence importance, frequency of terms, etc. For abstractive summarisation, use techniques like word embeddings.
3. **Model Training**: Train a model to understand the structure and content of the text. This could be a simple model for extractive summarisation or a complex neural network for abstractive summarisation.
4. **Generation**: Generate the summary by selecting key sentences (extractive) or generating new sentences (abstractive).



In [ ]:
# Step to install the transformers library
!pip install transformers

In [ ]:
# Import the necessary library
from transformers import pipeline

# Load a summarisation pipeline
summarizer = pipeline("summarization")

def summarize_text(text):
    """Use the summarizer to condense the text"""
    try:
        summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return text  # Return original text if summarization fails

# Apply summarization to the 'clean_content' column
df['summary_transformer'] = df['clean_content'].apply(summarize_text)

print("\nFirst few rows of the summarized dataset:")
print(df[['clean_content', 'summary_transformer']].head())


In [ ]:
import gdown
import zipfile
import os
url = 'https://drive.google.com/uc?id=1J0U-Ho_qH73wtjRYjDSKzaDT274zdljV'
output = 'Goldstandard.zip'
gdown.download(url, output, quiet=False)


if zipfile.is_zipfile(output):
    with zipfile.ZipFile(output, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    print("Files extracted:")
    print(os.listdir('/content/'))
else:
    print("Downloaded file is not a zip file.")

### Combining Source Texts and Gold Standard Summaries

In this section, we will combine source texts and their corresponding gold standard summaries into a single DataFrame, perform basic preprocessing on the text content, and prepare the data for further analysis.

1. **Directory Paths**:
   - Define paths for source texts (`source_dir`) and gold standard summaries (`gold_summary_dir`).

2. **Combine Files into a DataFrame**:
   - `combine_files_to_dataframe(directory_path)`: Reads files from the specified directory and combines their contents into a DataFrame.

3. **Load Source Texts and Gold Standard Summaries**:
   - Load source texts into `df_source` and remove the '.english' suffix from titles.
   - Load gold standard summaries into `df_gold`.

4. **Merge DataFrames**:
   - Merge `df_source` and `df_gold` on 'Title', creating a combined DataFrame with suffixes `('_source', '_gold')`.

5. **Select a Subset of Data**:
   - Select the first 10 rows for analysis.

6. **Preprocess Text**:
   - `clean_text(text)`: Cleans text by removing HTML tags, extra spaces, standalone numbers, and leading/trailing whitespace.

7. **Apply Preprocessing**:
   - Apply `clean_text` to the 'Content_source' column and store the result in 'clean_content'.



In [ ]:
import pandas as pd
import os
from transformers import pipeline

# Directory paths
source_dir = '/content/SourceTexts/english'
gold_summary_dir = '/content/Goldstandard'


# Function to combine files into a DataFrame
def combine_files_to_dataframe(directory_path):
    data = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                content = file.read()
                data.append({'Title': file_name, 'Content': content.strip()})
    df = pd.DataFrame(data)
    return df

# Load source texts
df_source = combine_files_to_dataframe(source_dir)
df_source['Title'] = df_source['Title'].str.replace('.english', '', regex=False)

# Load gold standard summaries
df_gold = combine_files_to_dataframe(gold_summary_dir)

# Merge dataframes on Title
df = pd.merge(df_source, df_gold, on='Title', suffixes=('_source', '_gold'))

# Select the first 10 files
df = df.head(10)

# Preprocess text function (clean HTML tags, remove stop words, etc.)
def clean_text(text):
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = text.strip()
    return text

# Apply preprocessing to the 'Content_source' column
df['clean_content'] = df['Content_source'].apply(clean_text)



### Define summarisation Functions

1. **summarise Text Using Sumy (LSA)**:
   - `summarize_text_sumy(text, sentences_count=2)`: Uses the LSA summarizer from the `sumy` package to generate a summary with a specified number of sentences.

2. **Generate Extractive Summary Using TF-IDF**:
   - `extractive_summary(text, top_n=3)`: Uses TF-IDF to score sentences based on term frequency-inverse document frequency and selects the top N sentences for the summary.

3. **summarise Text Using Summa (TextRank)**:
   - `summarize_text_summa(text)`: Uses the `summa` library to generate a summary based on the TextRank algorithm.

4. **summarise Text Using Transformer**:
   - Load the summarization pipeline: `transformer_summarizer = pipeline("summarization")`.
   - `summarize_text_transformer(text)`: Uses a pre-trained transformer model to generate a summary, with constraints on the maximum and minimum length of the summary.





In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from summa import summarizer as summa_summarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from transformers import pipeline

# Function to summarize text using Sumy
def summarize_text_sumy(text, sentences_count=2):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count)
    return ' '.join([str(sentence) for sentence in summary])

# Function to generate extractive summary using TF-IDF
def extractive_summary(text, top_n=3):
    sentences = text.split('.')
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    sentence_scores = np.sum(X.toarray(), axis=1)
    top_sentence_indices = sentence_scores.argsort()[-top_n:][::-1]
    summary = ' '.join([sentences[i] for i in top_sentence_indices])
    return summary

# Function to summarize text using Summa
def summarize_text_summa(text):
    summary = summa_summarizer.summarize(text)
    return summary

# Load the summarization pipeline for Transformer
transformer_summarizer = pipeline("summarization")

# Function to summarize text using Transformer
def summarize_text_transformer(text):
    try:
        summary = transformer_summarizer(text, max_length=50, min_length=25, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return text

# Generate summaries
df['summary_sumy'] = df['clean_content'].apply(lambda x: summarize_text_sumy(x, sentences_count=2))
df['summary_tfidf'] = df['clean_content'].apply(extractive_summary)
df['summary_summa'] = df['clean_content'].apply(summarize_text_summa)
df['summary_transformer'] = df['clean_content'].apply(summarize_text_transformer)



> **Now we can visualise the summaries in a table using Plotly, allowing us to view the content summarisation results side by side for easy comparison.**






In [ ]:
from tabulate import tabulate
import plotly.graph_objects as go
# Visualize the data using Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=df.columns[:8],
                fill_color='paleturquoise',
align='left'),
    cells=dict(values=[df['Title'], df['Content_source'], df['Content_gold'],df['clean_content'],df['summary_sumy'],df['summary_tfidf'],df['summary_summa'], df['summary_transformer']],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(
    title='Content Summarization',
    width=1000,
    height=600,
)

fig.show()

> **we can also displays sample summaries from each method, allowing for a quick comparison of original content and generated summaries.**





In [ ]:
# Display sample summaries from each method
sample_indices = [0, 1, 2]  # Adjust these indices to sample different rows

for i in sample_indices:
    print(f"Original Content [{i}]:\n{df['clean_content'][i]}\n")
    print(f"Sumy Summary [{i}]:\n{df['summary_sumy'][i]}\n")
    print(f"Summa Summary [{i}]:\n{df['summary_summa'][i]}\n")
    print(f"TFIDF Summary [{i}]:\n{df['summary_tfidf'][i]}\n")
    print(f"Transformer Summary [{i}]:\n{df['summary_transformer'][i]}\n")
    print("="*80)


The following sections analyse and visualize different aspects of text summarisation methods. We will conduct four main analyses:

1. **Trend Analysis of Summary Lengths:** Examine the average lengths of summaries generated by different methods.
2. **Word Cloud Visualization:** Visualize the most frequent words in the summaries.
3. **Readability Scores:** Calculate and compare the readability of summaries.
4. **Semantic Similarity:** Measure the semantic similarity between the original text and the summaries.


## 1. Trend Analysis of Summary Lengths
To compare the average length of the original texts with the lengths of summaries produced by various summarisation methods. This helps in understanding how concise each method is in summarising the content.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv('summarised_data.csv')

# Calculating the lengths of the original and summarised texts
df['original_length'] = df['clean_content'].apply(lambda x: len(x.split()))
df['summary_sumy_length'] = df['summary_sumy'].apply(lambda x: len(x.split()))
df['summary_summa_length'] = df['summary_summa'].apply(lambda x: len(x.split()))
df['summary_tfidf_length'] = df['summary_tfidf'].apply(lambda x: len(x.split()))
df['summary_transformer_length'] = df['summary_transformer'].apply(lambda x: len(x.split()))

# Calculate average lengths
average_lengths = {
    'Original': df['original_length'].mean(),
    'Sumy': df['summary_sumy_length'].mean(),
    'Summa': df['summary_summa_length'].mean(),
    'TFIDF': df['summary_tfidf_length'].mean(),
    'Transformer': df['summary_transformer_length'].mean()
}

# Create a DataFrame for plotting
average_lengths_df = pd.DataFrame(list(average_lengths.items()), columns=['Method', 'Average Length'])

# Plotting the trend
plt.figure(figsize=(14, 8))
sns.lineplot(data=average_lengths_df, x='Method', y='Average Length', marker='o')
plt.title('Trend of Average Text Lengths Across summarisation Methods')
plt.xlabel('summarisation Method')
plt.ylabel('Average Length of Text')
plt.show()


## 2. Word Cloud

To visualize the most frequent words in the summaries generated by different methods, providing insight into the key terms emphasized by each summarisation technique.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
summarisation_methods = ['summary_sumy', 'summary_summa', 'summary_tfidf']
# Generate word clouds for each summarisation method
for method in summarisation_methods:
    text = ' '.join(df[method])
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud for {method}')
    plt.axis('off')
    plt.show()


## 3. Readability Scores
To calculate and compare the readability of summaries generated by different methods. Readability scores indicate how easy it is to read and understand the text.

nstall and Import textstat:
Install the `textstat` package for calculating readability scores.





In [ ]:
!pip install textstat

In [ ]:
from textstat import textstat
from sklearn.metrics.pairwise import cosine_similarity
import pprint
# Function to calculate readability scores
def calculate_readability_scores(df, summarisation_methods):
    readability_scores = {}
    for method in summarisation_methods:
        method_scores = []
        for summary in df[method]:
            score = textstat.flesch_reading_ease(summary)
            method_scores.append(score)
        avg_score = sum(method_scores) / len(method_scores)
        readability_scores[method] = avg_score
    return readability_scores

readability_scores = calculate_readability_scores(df, summarisation_methods)

# Print readability scores
pprint.pprint(readability_scores)


## 4. Semantic Similarity

To measure the cosine similarity between the original text and the summaries, providing insight into how well the summaries capture the meaning of the original content.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pprint
# Function to calculate semantic similarity
def calculate_semantic_similarity(df, summarisation_methods):
    vectorizer = TfidfVectorizer()
    similarities = {}
    for method in summarisation_methods:
        method_similarities = []
        for i, row in df.iterrows():
            try:
                vectors = vectorizer.fit_transform([row['clean_content'], row[method]])
                sim = cosine_similarity(vectors[0:1], vectors[1:2])
                method_similarities.append(sim[0][0])
            except:
                pass
        avg_similarity = sum(method_similarities) / len(method_similarities)
        similarities[method] = avg_similarity
    return similarities

semantic_similarities = calculate_semantic_similarity(df, summarisation_methods)

# Print semantic similarity scores
pprint.pprint(semantic_similarities)


### **5-Evaluation**:

### 1. **BLEU (Bilingual Evaluation Understudy)**
- Originally designed for machine translation but can be used for summarisation.
- Measures the precision of n-grams in the generated summary against the reference summary.

### 2. **METEOR (Metric for Evaluation of Translation with Explicit ORdering)**
- Another metric from the machine translation community.
- Considers synonymy and stemming, providing a more nuanced evaluation than BLEU.

### 3. **CIDEr (Consensus-based Image Description Evaluation)**
- Used mainly in image captioning, but applicable to summarisation.
- Measures the consensus between generated and reference summaries using Term Frequency-Inverse Document Frequency (TF-IDF) weighting.

### 4. **BERTScore**
- Leverages pre-trained BERT embeddings to evaluate summaries.
- Measures the similarity between the embeddings of the generated and reference summaries.

### 5. **Human Evaluation**
- **Fluency**: Is the summary grammatically correct and well-structured?
- **Relevance**: Does the summary include the most important points of the original text?
- **Coherence**: Is the summary logically organized?
- **Informativeness**: Does the summary provide enough information?

### 6. **summarisation-Specific Metrics**
- **Pyramid Method**: Involves human annotators identifying and scoring key content units in the summary and reference texts.
- **Content Responsiveness**: Evaluates how well the summary responds to specific questions related to the source text.


### **5.1 ROUGE Scores**
To evaluate the quality of summaries using ROUGE scores, which compare the overlap of n-grams between the generated summaries and reference texts.

1. Install and Import rouge-score:

Install the `rouge-score` package for calculating ROUGE scores.

In [ ]:
!pip install rouge-score  # Install the rouge-score package

2. Calculate ROUGE Scores:

Define a function to compute ROUGE scores and apply it to the summaries.

In [ ]:
from rouge_score import rouge_scorer
# Function to calculate ROUGE scores
def calculate_rouge_scores(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

# Evaluate summaries
df['rouge_sumy'] = df.apply(lambda row: calculate_rouge_scores(row['Content_gold'], row['summary_sumy']), axis=1)
df['rouge_tfidf'] = df.apply(lambda row: calculate_rouge_scores(row['Content_gold'], row['summary_tfidf']), axis=1)
df['rouge_summa'] = df.apply(lambda row: calculate_rouge_scores(row['Content_gold'], row['summary_summa']), axis=1)
df['rouge_transformer'] = df.apply(lambda row: calculate_rouge_scores(row['Content_gold'], row['summary_transformer']), axis=1)


3. Extract and Plot ROUGE Scores:
Extract F1 scores for ROUGE-1, ROUGE-2, and ROUGE-L, and plot them.

In [ ]:
# Extract ROUGE-1 F1 scores
df['rouge_sumy_1'] = df['rouge_sumy'].apply(lambda x: x['rouge1'].fmeasure)
df['rouge_tfidf_1'] = df['rouge_tfidf'].apply(lambda x: x['rouge1'].fmeasure)
df['rouge_summa_1'] = df['rouge_summa'].apply(lambda x: x['rouge1'].fmeasure)
df['rouge_transformer_1'] = df['rouge_transformer'].apply(lambda x: x['rouge1'].fmeasure)

# Extract ROUGE-2 F1 scores
df['rouge_sumy_2'] = df['rouge_sumy'].apply(lambda x: x['rouge2'].fmeasure)
df['rouge_tfidf_2'] = df['rouge_tfidf'].apply(lambda x: x['rouge2'].fmeasure)
df['rouge_summa_2'] = df['rouge_summa'].apply(lambda x: x['rouge2'].fmeasure)
df['rouge_transformer_2'] = df['rouge_transformer'].apply(lambda x: x['rouge2'].fmeasure)

# Plot ROUGE-1 scores
plt.figure(figsize=(10, 6))
plt.plot(df['Title'], df['rouge_sumy_1'], label='Sumy ROUGE-1', marker='o')
plt.plot(df['Title'], df['rouge_tfidf_1'], label='TF-IDF ROUGE-1', marker='o')
plt.plot(df['Title'], df['rouge_summa_1'], label='Summa ROUGE-1', marker='o')
plt.plot(df['Title'], df['rouge_transformer_1'], label='Transformer ROUGE-1', marker='o')
plt.xlabel('Document')
plt.ylabel('ROUGE-1 F1 Score')
plt.legend()
plt.xticks(rotation=90)
plt.title('ROUGE-1 F1 Scores for Different summarisation Methods')
plt.show()

# Plot ROUGE-2 scores
plt.figure(figsize=(10, 6))
plt.plot(df['Title'], df['rouge_sumy_2'], label='Sumy ROUGE-2', marker='o')
plt.plot(df['Title'], df['rouge_tfidf_2'], label='TF-IDF ROUGE-2', marker='o')
plt.plot(df['Title'], df['rouge_summa_2'], label='Summa ROUGE-2', marker='o')
plt.plot(df['Title'], df['rouge_transformer_2'], label='Transformer ROUGE-2', marker='o')
plt.xlabel('Document')
plt.ylabel('ROUGE-2 F1 Score')
plt.legend()
plt.xticks(rotation=90)
plt.title('ROUGE-2 F1 Scores for Different summarisation Methods')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Extract ROUGE-1, ROUGE-2, and ROUGE-L F1 scores
df['rouge_sumy_1'] = df['rouge_sumy'].apply(lambda x: x['rouge1'].fmeasure)
df['rouge_tfidf_1'] = df['rouge_tfidf'].apply(lambda x: x['rouge1'].fmeasure)
df['rouge_transformer_1'] = df['rouge_transformer'].apply(lambda x: x['rouge1'].fmeasure)

df['rouge_sumy_2'] = df['rouge_sumy'].apply(lambda x: x['rouge2'].fmeasure)
df['rouge_tfidf_2'] = df['rouge_tfidf'].apply(lambda x: x['rouge2'].fmeasure)
df['rouge_transformer_2'] = df['rouge_transformer'].apply(lambda x: x['rouge2'].fmeasure)

df['rouge_sumy_L'] = df['rouge_sumy'].apply(lambda x: x['rougeL'].fmeasure)
df['rouge_tfidf_L'] = df['rouge_tfidf'].apply(lambda x: x['rougeL'].fmeasure)
df['rouge_transformer_L'] = df['rouge_transformer'].apply(lambda x: x['rougeL'].fmeasure)

# Plot ROUGE-1, ROUGE-2, and ROUGE-L scores

# Bar Chart of Average ROUGE Scores
average_rouge_scores = {
    'Sumy ROUGE-1': df['rouge_sumy_1'].mean(),
    'TF-IDF ROUGE-1': df['rouge_tfidf_1'].mean(),
    'Transformer ROUGE-1': df['rouge_transformer_1'].mean(),
    'Sumy ROUGE-2': df['rouge_sumy_2'].mean(),
    'TF-IDF ROUGE-2': df['rouge_tfidf_2'].mean(),
    'Transformer ROUGE-2': df['rouge_transformer_2'].mean(),
    'Sumy ROUGE-L': df['rouge_sumy_L'].mean(),
    'TF-IDF ROUGE-L': df['rouge_tfidf_L'].mean(),
    'Transformer ROUGE-L': df['rouge_transformer_L'].mean()
}

# Convert to DataFrame for easy plotting
avg_rouge_df = pd.DataFrame(list(average_rouge_scores.items()), columns=['Metric', 'Average Score'])

plt.figure(figsize=(12, 8))
plt.bar(avg_rouge_df['Metric'], avg_rouge_df['Average Score'], color=['blue', 'green', 'red', 'blue', 'green', 'red', 'blue', 'green', 'red'])
plt.ylabel('Average ROUGE Score')
plt.title('Average ROUGE Scores for Different summarisation Methods')
plt.xticks(rotation=45)
plt.show()

# Scatter Plots of Individual Document Scores
plt.figure(figsize=(12, 8))
plt.scatter(df['Title'], df['rouge_sumy_1'], label='Sumy ROUGE-1', marker='o')
plt.scatter(df['Title'], df['rouge_tfidf_1'], label='TF-IDF ROUGE-1', marker='x')
plt.scatter(df['Title'], df['rouge_transformer_1'], label='Transformer ROUGE-1', marker='^')
plt.xlabel('Document')
plt.ylabel('ROUGE-1 F1 Score')
plt.legend()
plt.xticks(rotation=90)
plt.title('ROUGE-1 F1 Scores for Different summarisation Methods')
plt.show()

plt.figure(figsize=(12, 8))
plt.scatter(df['Title'], df['rouge_sumy_2'], label='Sumy ROUGE-2', marker='o')
plt.scatter(df['Title'], df['rouge_tfidf_2'], label='TF-IDF ROUGE-2', marker='x')
plt.scatter(df['Title'], df['rouge_transformer_2'], label='Transformer ROUGE-2', marker='^')
plt.xlabel('Document')
plt.ylabel('ROUGE-2 F1 Score')
plt.legend()
plt.xticks(rotation=90)
plt.title('ROUGE-2 F1 Scores for Different summarisation Methods')
plt.show()

plt.figure(figsize=(12, 8))
plt.scatter(df['Title'], df['rouge_sumy_L'], label='Sumy ROUGE-L', marker='o')
plt.scatter(df['Title'], df['rouge_tfidf_L'], label='TF-IDF ROUGE-L', marker='x')
plt.scatter(df['Title'], df['rouge_transformer_L'], label='Transformer ROUGE-L', marker='^')
plt.xlabel('Document')
plt.ylabel('ROUGE-L F1 Score')
plt.legend()
plt.xticks(rotation=90)
plt.title('ROUGE-L F1 Scores for Different summarisation Methods')
plt.show()



- **Sumy:**
  - ROUGE-1: ~0.12
  - ROUGE-2: ~0.025
  - ROUGE-L: ~0.08
- **TF-IDF:**
  - ROUGE-1: ~0.20
  - ROUGE-2: ~0.04
  - ROUGE-L: ~0.11
- **Transformer:**
  - ROUGE-1: ~0.10
  - ROUGE-2: ~0.025
  - ROUGE-L: ~0.07

### Analysis
1. **ROUGE-1:**
   - **TF-IDF** has the highest average ROUGE-1 score (~0.20), indicating it captures the most unigram (single word) matches compared to the other methods.

2. **ROUGE-2:**
   - **TF-IDF** again has the highest average ROUGE-2 score (~0.04), suggesting it captures the most bigram (two consecutive words) matches.

3. **ROUGE-L:**
   - **TF-IDF** has the highest average ROUGE-L score (~0.11), which measures the longest common subsequence and indicates it captures the structure of the original text better.

Based on the average ROUGE scores, the **TF-IDF summarizer** appears to be the best performer among the three methods (Sumy, TF-IDF, and Transformer). It consistently has the highest scores across ROUGE-1, ROUGE-2, and ROUGE-L metrics.


### **5.1 Bert Score**

evaluate the quality of generated summaries using BERTScore, a state-of-the-art metric that leverages BERT embeddings to measure the similarity between the reference and generated texts. BERTScore provides a more nuanced understanding of text similarity compared to traditional metrics.

In [ ]:
!pip install bert-score

In [ ]:

from bert_score import score

def calculate_bertscore(reference_texts, generated_texts):
    P, R, F1 = score(generated_texts, reference_texts, lang="en", verbose=True)
    return P.mean().item(), R.mean().item(), F1.mean().item()

# Extract reference and generated summaries
reference_texts = df['Content_gold'].tolist()
generated_sumy = df['summary_sumy'].tolist()
generated_tfidf = df['summary_tfidf'].tolist()
generated_transformer = df['summary_transformer'].tolist()

# Calculate BERTScores
bertscore_sumy = calculate_bertscore(reference_texts, generated_sumy)
bertscore_tfidf = calculate_bertscore(reference_texts, generated_tfidf)
bertscore_transformer = calculate_bertscore(reference_texts, generated_transformer)

print(f"BERTScore (Sumy): Precision={bertscore_sumy[0]}, Recall={bertscore_sumy[1]}, F1={bertscore_sumy[2]}")
print(f"BERTScore (TF-IDF): Precision={bertscore_tfidf[0]}, Recall={bertscore_tfidf[1]}, F1={bertscore_tfidf[2]}")
print(f"BERTScore (Transformer): Precision={bertscore_transformer[0]}, Recall={bertscore_transformer[1]}, F1={bertscore_transformer[2]}")


